In [1]:
import numpy as np
import scipy.linalg as la

from qiskit import execute
from qiskit import Aer
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer.noise import NoiseModel

from racbem import *

import os
import pickle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/envs/quantum/lib/python3.7/site-packages/qiskit/validation/fields/custom.py:76: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.integer, numpy.float,
/Users/flynn_chen/.pyxbld/temp.macosx-10.9-x86_64-3.7/pyrex/qutip/cy/openmp/parfuncs.cpp:636:10: fatal error: 'src/zspmv_openmp.hpp' file not found
#include "src/zspmv_openmp.hpp"
         ^~~~~~~~~~~~~~~~~~~~~~
1 error generated.


In [2]:
def GetBackend(backend_name=None):
    if backend_name == None:
        backend = Aer.get_backend('qasm_simulator')
    else:
        provider = IBMQ.load_account()
        backend = provider.get_backend(backend_name)
    return backend

To get consistent output:
1. remove random noise (sigma=0)
2. set seed = 0, and added seed in the build_random_circuit

In [3]:
backend_name = 'ibmq_burlington'
#backend_name = None
kappa = 5                   # condition number
n_sys_qubit = 3             # the number of system qubits
n_be_qubit = 1              # the number of block-encoding qubit
n_sig_qubit = 1             # the number of signal qubit
n_tot_qubit = n_sig_qubit+n_be_qubit+n_sys_qubit
n_depth = 15                # the depth of random circuit
prob_one_q_op = 0.5         # the probability of selecting a one-qubit
                            # operation when two_q_op is allowed
basis_gates = ['u1','u2','cx']
digit_shots = 13
n_shots = 2**digit_shots    # the number of shots used in measurements
sigma = 1.0                 # parameter used to rescale noise model
# state |0^n>
b = np.zeros((2**n_sys_qubit,))
b[0] = 1.0
load_architecture = True    # True:     load architure locally
                            # False:    need to save an IBM account beforehand

In [4]:
seed = 0
np.random.seed(seed)

# instances of RACBEM
be = BlockEncoding(n_be_qubit, n_sys_qubit)
qsp = QSPCircuit(n_sig_qubit, n_be_qubit, n_sys_qubit)

# retrieve backends and architectures
backend = GetBackend()
if load_architecture:
    if os.path.exists(backend_name+'_backend_config.pkl'):
        noise_backend = pickle.load(open(backend_name+'_backend_config.pkl','rb'))
        noise_model = NoiseModel.from_dict(noise_backend['noise_dict'])
        coupling_map = noise_backend['coupling_map']
        tot_q_device = noise_backend['tot_q_device']
        print("\nload architecture locally at: %s_backend_config.pkl\n"%(backend_name))
    else:
        raise Exception("no locally saved architecture: %s_backend_config.pkl"%(backend_name), load_architecture)
else:
    noise_backend = GetBackend(backend_name=backend_name)
    coupling_map = noise_backend.configuration().coupling_map
    noise_model = NoiseModel.from_backend(noise_backend)
    tot_q_device = noise_backend.configuration().n_qubits
    pickle.dump({'noise_dict': noise_model.to_dict(), 'coupling_map': coupling_map, 'tot_q_device': tot_q_device, 
                'basis_gates': noise_backend.configuration().basis_gates}, open(backend_name+'_backend_config.pkl','wb'))
    print("retrieve architecture from IBM Q and save locally at: %s_backend_config.pkl\n"%(backend_name))
assert tot_q_device >= n_tot_qubit
new_noise_model = scale_noise_model(noise_model, sigma)

# exclude qubit 0 as signal qubit, shift the remaining labels by -1
be_map = [[q[0]-1,q[1]-1] for q in coupling_map if (0 not in q) and 
        (q[0] < n_tot_qubit) and (q[1] < n_tot_qubit)]
be.build_random_circuit(n_depth, basis_gates=basis_gates, 
        prob_one_q_op=prob_one_q_op, coupling_map=be_map, seed=seed)
be.build_dag()

# load phase factors
data = np.loadtxt("phi_inv_%d.txt"%(kappa))
phi_seq = data[:-2]
#phi_seq = np.random.rand(7) * np.pi
#phi_seq = np.pi/(2**np.arange(0, 15))
scale_fac = data[-2]
app_err = data[-1]

# retrieve block-encoded matrix
UA = retrieve_unitary_matrix(be.qc)
A = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
(svd_U, svd_S, svd_VH) = la.svd(A)
print("kappa=%d, sigma=%.2f, polynomial approximation error=%.3e"%(kappa, sigma, app_err))
print("")
print("Generic RACBEM")
print("singular value (A) = \n", np.around(svd_S, decimals=3))

# succ prob via measurement
qsp.build_circuit(be.qc, be.qc_dag, phi_seq, realpart=True, measure=True)
compiled_circ = qsp.qcircuit
job = execute(compiled_circ, backend=backend,
        noise_model=new_noise_model, shots=n_shots)
job_monitor(job)
result = job.result()
counts = result.get_counts(compiled_circ)
# both the signal and the ancilla qubit for block-encoding needs to
# be 0
prob_meas = np.float(counts['00']) / n_shots
# succ prob via noiseless simulator
qsp.build_circuit(be.qc, be.qc_dag, phi_seq, realpart=True, measure=False)
state = retrieve_state(qsp.qcircuit)
x = state[0:2**n_sys_qubit]
prob_qsp = la.norm(x)**2
# exact succ prob
svd_S_herm = (1-1.0/kappa)*svd_S**2+1.0/kappa
A_herm_inv = svd_VH.transpose().conjugate() @ np.diag(1/svd_S_herm) @ svd_VH
x_exact = A_herm_inv @ b / scale_fac
prob_exact = la.norm(x_exact)**2
print("succ prob (exact)     = ", prob_exact)
print("succ prob (noiseless) = ", prob_qsp)
print("succ prob (measure)   = ", prob_meas)
print("")

# instance of Hermitian Block-Encoding
n_be_qubit = n_be_qubit + 1     # add one extra qubit as sig_qubit in quadratic QSVT
be = Hermitian_BlockEncoding(n_be_qubit, n_sys_qubit)
be.set_cndnum(kappa)
be.build_random_circuit(n_depth, basis_gates=basis_gates, 
        prob_one_q_op=prob_one_q_op, coupling_map=be_map)
be.build_dag()
UA = retrieve_unitary_matrix(be.qc)
A = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
UA = retrieve_unitary_matrix(be.qc_dag)
A_dag = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
(svd_U, svd_S, svd_VH) = la.svd(A)
print("Hermitian RACBEM")
print("singular value (A) = \n", np.around(svd_S, decimals=3))
print("condition number (A)  = %.3f"%(svd_S.max()/svd_S.min()))
print("||A - A^\dagger||_2   = %.3e"%(la.norm(A - A_dag)))


load architecture locally at: ibmq_burlington_backend_config.pkl

kappa=5, sigma=1.00, polynomial approximation error=1.902e-02

Generic RACBEM
singular value (A) = 
 [0.893 0.893 0.795 0.795 0.49  0.49  0.477 0.477]
Job Status: job has successfully run
succ prob (exact)     =  0.1485847611577349
succ prob (noiseless) =  0.1484394470037316
succ prob (measure)   =  0.17431640625

Hermitian RACBEM
singular value (A) = 
 [0.837 0.837 0.705 0.705 0.392 0.392 0.382 0.382]
condition number (A)  = 2.193
||A - A^\dagger||_2   = 6.825e-15


In [5]:
A

array([[ 0.5023581 -6.10622664e-16j,  0.00616124-2.00119355e-02j,
        -0.01023391+7.71643612e-03j,  0.0073908 +2.27546487e-03j,
         0.05287521+3.46458058e-02j, -0.00175457-3.81002916e-02j,
         0.11205225-1.44740011e-02j, -0.10316353+4.75082510e-03j],
       [ 0.00616124+2.00119355e-02j,  0.5023581 -1.27675648e-15j,
        -0.0073908 +2.27546487e-03j, -0.01023391+7.71643612e-03j,
         0.02287999+3.05158483e-02j,  0.05287521+3.46458058e-02j,
         0.08262726-6.19517605e-02j,  0.11205225-1.44740011e-02j],
       [-0.01023391-7.71643612e-03j, -0.0073908 -2.27546487e-03j,
         0.50805776-5.55111512e-16j, -0.00616124+2.00119355e-02j,
         0.10255336-4.09318147e-02j,  0.10316353-4.75082510e-03j,
        -0.01876417+6.03657688e-02j,  0.00175457+3.81002916e-02j],
       [ 0.0073908 -2.27546487e-03j, -0.01023391-7.71643612e-03j,
        -0.00616124-2.00119355e-02j,  0.50805776-1.22124533e-15j,
        -0.08262726+6.19517605e-02j,  0.10255336-4.09318147e-02j,
       

In [6]:
A_dag

array([[ 0.5023581 -8.32667268e-16j,  0.00616124-2.00119355e-02j,
        -0.01023391+7.71643612e-03j,  0.0073908 +2.27546487e-03j,
         0.05287521+3.46458058e-02j, -0.00175457-3.81002916e-02j,
         0.11205225-1.44740011e-02j, -0.10316353+4.75082510e-03j],
       [ 0.00616124+2.00119355e-02j,  0.5023581 -1.05471187e-15j,
        -0.0073908 +2.27546487e-03j, -0.01023391+7.71643612e-03j,
         0.02287999+3.05158483e-02j,  0.05287521+3.46458058e-02j,
         0.08262726-6.19517605e-02j,  0.11205225-1.44740011e-02j],
       [-0.01023391-7.71643612e-03j, -0.0073908 -2.27546487e-03j,
         0.50805776-1.16573418e-15j, -0.00616124+2.00119355e-02j,
         0.10255336-4.09318147e-02j,  0.10316353-4.75082510e-03j,
        -0.01876417+6.03657688e-02j,  0.00175457+3.81002916e-02j],
       [ 0.0073908 -2.27546487e-03j, -0.01023391-7.71643612e-03j,
        -0.00616124-2.00119355e-02j,  0.50805776-1.38777878e-15j,
        -0.08262726+6.19517605e-02j,  0.10255336-4.09318147e-02j,
       